In [1]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import requests
import zipfile
import os

## Baixando os dados referentes ao bolsa família.

In [ ]:
response = requests.get("https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/novo-bolsa-familia/202311_NovoBolsaFamilia.zip")

In [ ]:
with open("bolsafamilia.zip", 'wb') as f:
    f.write(response.content)

### Descompactando...

In [ ]:
with zipfile.ZipFile("bolsafamilia.zip", 'r') as zip_ref:
    zip_ref.extractall('.')

### Baixando os dados do IBGE para obter a ocupação por estado

In [3]:
response = requests.get("https://ftp.ibge.gov.br/Indicadores_Sociais/Sintese_de_Indicadores_Sociais/Sintese_de_Indicadores_Sociais_2023/xls/1_Estrutura_economica_e_mercado_de_trabalho_xls.zip")

In [8]:
with open("ibge.zip", 'wb') as f:
    f.write(response.content)

In [9]:
# Criar o diretório de destino se ele não existir
if not os.path.exists('./ibge'):
    os.makedirs('./ibge')

In [10]:
with zipfile.ZipFile("ibge.zip", 'r') as zip_ref:
    zip_ref.extractall('./ibge')

In [12]:
response = requests.get("https://www.gov.br/receitafederal/pt-br/acesso-a-informacao/dados-abertos/receitadata/arrecadacao/arrecadacao-por-estado/arrecadacao-uf-2021/arrecadacao-por-uf-internet-dezembro-2021.ods/@@download/file")

In [13]:
with open("arrecadacao.ods", 'wb') as f:
    f.write(response.content)

In [17]:
## Obtendo os dados de transferências de recursos para os Estados

In [2]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

In [3]:
driver.get('https://portaldatransparencia.gov.br/localidades?ano=2021')

In [4]:
tabela_html = driver.find_element(By.CLASS_NAME, 'tabelaPrimeiroNivel').get_attribute('outerHTML')

In [5]:
driver.quit()

In [6]:
## iniciando o processo de limpeza dos dados

In [7]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [8]:
df_bolsa_familia = pd.read_csv('202311_NovoBolsaFamilia.csv', encoding = "ISO-8859-1", sep = ';')

In [9]:
df_total_bolsa_familia = df_bolsa_familia[df_bolsa_familia['MÊS REFERÊNCIA'] == 202311]

In [10]:
df_total_bolsa_familia = df_total_bolsa_familia['UF'].value_counts()

In [11]:
df_total_bolsa_familia = df_total_bolsa_familia.reset_index()

In [12]:
df_total_bolsa_familia.columns = ['Estado', 'Nº de beneficiários do bolsa família']

In [13]:
df_ocupados = pd.read_excel('./ibge/Tabela 1.12 (Pos_Geo).xls', header=7, nrows=33)

In [14]:
df_ocupados = df_ocupados.rename(columns={'Unnamed: 0': 'Estado', 'Unnamed: 1': 'Nº de ocupados'})

In [15]:
df_ocupados = df_ocupados[["Estado", "Nº de ocupados"]]

In [16]:
df = df_ocupados.drop([0,1,9,19,24,28])

In [17]:
df = df.reset_index(drop=True)

In [18]:
df['Nº de ocupados'] = df['Nº de ocupados'].astype(float)*1000

In [19]:
nome_para_sigla = {
    'Acre': 'AC',
    'Alagoas': 'AL',
    'Amapá': 'AP',
    'Amazonas': 'AM',
    'Bahia': 'BA',
    'Ceará': 'CE',
    'Distrito Federal': 'DF',
    'Espírito Santo': 'ES',
    'Goiás': 'GO',
    'Maranhão': 'MA',
    'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG',
    'Pará': 'PA',
    'Paraíba': 'PB',
    'Paraná': 'PR',
    'Pernambuco': 'PE',
    'Piauí': 'PI',
    'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO',
    'Roraima': 'RR',
    'Santa Catarina': 'SC',
    'São Paulo': 'SP',
    'Sergipe': 'SE',
    'Tocantins': 'TO'
}


In [20]:
df['Estado'] = df['Estado'].map(nome_para_sigla)


In [21]:
df_combinado = pd.merge(df_total_bolsa_familia, df, on='Estado')

In [22]:
df_combinado["Beneficiários Bolsa Família/Ocupados"] = df_combinado['Nº de beneficiários do bolsa família']/df_combinado["Nº de ocupados"]

In [23]:
df_arrecadacao = pd.read_excel('arrecadacao.ods', engine='odf', header = 7)

In [24]:
df_arrecadacao_tratado = df_arrecadacao.drop(range(0,36))

In [25]:
df_arrecadacao_tratado.drop("TOTAL", axis=1,  inplace=True)

In [26]:
df_arrecadacao_tratado.drop("RECEITAS ", axis=1,  inplace=True)

In [27]:
df_arrecadacao_tratado = df_arrecadacao_tratado.transpose()


In [28]:
df_arrecadacao_tratado = df_arrecadacao_tratado.reset_index()

In [29]:
df_arrecadacao_tratado.columns = ['Estado','Arrecadação em Reais']


In [30]:
df_arrecadacao_tratado['Arrecadação em Reais'] = df_arrecadacao_tratado['Arrecadação em Reais'].astype(float)

In [31]:
df_transferencias = pd.read_html(tabela_html)[0]

In [32]:
df_transferencias['Localidade']= df_transferencias['Localidade'].map(nome_para_sigla)

In [33]:
# Deixando apenas as colunas referentes ao total repassado para cada estado e excluindo linhas irrelevantes
df_transferencias = df_transferencias.drop(range(26,31))
df_transferencias.drop("Valor transferido diretamente ao Estado", axis=1, inplace=True)
df_transferencias.drop("Valor transferido aos Municípios", axis=1, inplace=True)


In [34]:
# Convertendo os valores para formato numérico para fazer a manipulação
df_transferencias['Valor total transferido'] = df_transferencias['Valor total transferido'].replace({'R\$': '', '\.': '', ',': '.'}, regex=True).astype(float)

In [35]:
# Dividindo o valor total por 12 para obter o valor mensal pela média
df_transferencias["Valor total transferido"] = df_transferencias["Valor total transferido"]/12

In [36]:
df_transferencias.rename(columns={'Localidade': 'Estado'}, inplace=True)

In [37]:
df_combinado = pd.merge(df_combinado, df_arrecadacao_tratado, on='Estado')

In [38]:
df_combinado = pd.merge(df_combinado, df_transferencias, on='Estado')

In [39]:
df_combinado["Transferências/Arrecadação"] = df_combinado['Valor total transferido']/df_combinado["Arrecadação em Reais"]

In [40]:
### apresentando os resultados

In [41]:
df_combinado

,Estado,Nº de beneficiários do bolsa família,Nº de ocupados,Beneficiários Bolsa Família/Ocupados,Arrecadação em Reais,Valor total transferido,Transferências/Arrecadação
0,SP,2449430,23500288.11834,0.10423,75802834639.89890,4192445724.29333,0.05531
1,BA,2384675,6022252.14790,0.39598,4562589340.23619,3145826650.61250,0.68948
2,RJ,1670742,7715755.22004,0.21654,32337936592.84400,4033407852.91833,0.12473
3,PE,1570046,3594104.22016,0.43684,3247227841.94345,1891740264.77917,0.58257
4,MG,1544179,10369730.36111,0.14891,12110506544.32390,3627730252.28500,0.29955
5,CE,1428899,3530589.85958,0.40472,3000599078.82148,2031635886.75583,0.67708
6,PA,1260245,3769340.22869,0.33434,1923668111.25154,2079177024.77250,1.08084
7,MA,1127943,2461736.08122,0.45819,1096377697.28324,1958095365.91750,1.78597
8,PB,660238,1447009.90388,0.45628,977523337.71247,1154894632.12833,1.18145
9,AM,633856,1686936.65229,0.37574,2042939897.01911,875270793.93417,0.42844


In [45]:
df_combinado.sort_values(by='Transferências/Arrecadação', ascending=False)[["Estado", "Transferências/Arrecadação"]].reset_index(drop=True)

,Estado,Transferências/Arrecadação
0,AP,3.57071
1,AC,2.25005
2,RR,2.24635
3,MA,1.78597
4,PI,1.73872
5,AL,1.40363
6,PB,1.18145
7,SE,1.17960
8,RN,1.13583
9,PA,1.08084


In [46]:
df_combinado.sort_values(by='Beneficiários Bolsa Família/Ocupados', ascending=False)[["Estado", "Beneficiários Bolsa Família/Ocupados"]].reset_index(drop=True)

,Estado,Beneficiários Bolsa Família/Ocupados
0,PI,0.46207
1,MA,0.45819
2,PB,0.45628
3,AL,0.44456
4,PE,0.43684
5,CE,0.40472
6,BA,0.39598
7,AC,0.38793
8,SE,0.38278
9,AM,0.37574
